    16_230109_100_kyungki_apt_이천
    
     
     data : /aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/002_apt_data/apt_의왕_이천_557.xls

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [3]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [4]:
#9 Folium를 활용한 공간 데이터 시각화
### 전국도시철도역사정보표준데이터 가져오기

apt_df = pd.read_excel("/aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/002_apt_data/apt_의왕_이천_557.xls")
apt_df.head()

,시군명,공동주택구분,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,의왕시,NaN,KT이자리에,오전동,333,2004-11-29,22,3,182,Y
1,의왕시,NaN,골드클래스,학의동,823-2,2019-05-03,16,8,420,N
2,의왕시,NaN,남양모란,오전동,358-19,1999-12-24,16,1,128,N
3,의왕시,NaN,뉴서울국화,오전동,838-3,1994-12-15,18,2,220,Y
4,의왕시,NaN,대명구름채,오전동,28,2004-12-21,23,6,518,Y


In [5]:
apt_df.drop(["공동주택구분"], axis=1,inplace=True)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,의왕시,KT이자리에,오전동,333,2004-11-29,22,3,182,Y
1,의왕시,골드클래스,학의동,823-2,2019-05-03,16,8,420,N
2,의왕시,남양모란,오전동,358-19,1999-12-24,16,1,128,N
3,의왕시,뉴서울국화,오전동,838-3,1994-12-15,18,2,220,Y
4,의왕시,대명구름채,오전동,28,2004-12-21,23,6,518,Y


In [6]:
apt_df['지번주소'] = apt_df['지번주소'].astype(str)

In [7]:
#okok
apt_df['addr'] = '경기도 ' + apt_df[['시군명', '읍면동주소','지번주소']].agg(' '.join, axis=1)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr
0,의왕시,KT이자리에,오전동,333,2004-11-29,22,3,182,Y,경기도 의왕시 오전동 333
1,의왕시,골드클래스,학의동,823-2,2019-05-03,16,8,420,N,경기도 의왕시 학의동 823-2
2,의왕시,남양모란,오전동,358-19,1999-12-24,16,1,128,N,경기도 의왕시 오전동 358-19
3,의왕시,뉴서울국화,오전동,838-3,1994-12-15,18,2,220,Y,경기도 의왕시 오전동 838-3
4,의왕시,대명구름채,오전동,28,2004-12-21,23,6,518,Y,경기도 의왕시 오전동 28


In [8]:
# https://sgisapi.kostat.go.kr/OpenAPI3/addr/stage.json

In [9]:
#5 seoul university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [10]:
#5 seoul university .. 주소 기반 위도,경도 추가
cnt = 0
for _, row in apt_df.iterrows():
    try:
        x, y = get_adress_x_y(row["addr"])
     
        apt_df.loc[_, "x"] = x
        apt_df.loc[_, "y"] = y
    except:
        print(row["addr"] + '은(는) 검색에 실패했습니다')
        cnt = cnt + 1

print("실패건수 : ", cnt)
apt_df.head()


경기도 의왕시 고천동 22433은(는) 검색에 실패했습니다
경기도 의왕시 오전동 44790은(는) 검색에 실패했습니다
경기도 의왕시 오전동 44223은(는) 검색에 실패했습니다
경기도 의정부시 낙양동 156-1은(는) 검색에 실패했습니다
경기도 의정부시 금오동 24473은(는) 검색에 실패했습니다
경기도 의정부시 의정부1동 44581은(는) 검색에 실패했습니다
경기도 의정부시 의정부동 44855은(는) 검색에 실패했습니다
경기도 의정부시 의정부동 44635은(는) 검색에 실패했습니다
경기도 의정부시 용현동 44840은(는) 검색에 실패했습니다
경기도 의정부시 의정부동 44834은(는) 검색에 실패했습니다
경기도 의정부시 신곡동 34213은(는) 검색에 실패했습니다
경기도 의정부시 의정부동 44638은(는) 검색에 실패했습니다
경기도 의정부시 의정부동 44614은(는) 검색에 실패했습니다
경기도 의정부시 의정부동 35827은(는) 검색에 실패했습니다
경기도 의정부시 낙양동 218-7은(는) 검색에 실패했습니다
경기도 의정부시 가능동 44757은(는) 검색에 실패했습니다
경기도 의정부시 낙양동 192은(는) 검색에 실패했습니다
경기도 의정부시 용현동 14580은(는) 검색에 실패했습니다
경기도 의정부시 용현동 12055은(는) 검색에 실패했습니다
경기도 의정부시 용현동 12114은(는) 검색에 실패했습니다
경기도 의정부시 장암동 12451은(는) 검색에 실패했습니다
경기도 의정부시 고산동 (입암로 13) 177은(는) 검색에 실패했습니다
경기도 의정부시 호원동 20180은(는) 검색에 실패했습니다
경기도 의정부시 호원동 44715은(는) 검색에 실패했습니다
경기도 의정부시 호원동 44835은(는) 검색에 실패했습니다
경기도 의정부시 호원동 44837은(는) 검색에 실패했습니다
경기도 의정부시 의정부동 44915은(는) 검색에 실패했습니다
경기도 이천시 송정동 454은(는) 검색에 실패했습니다
경기도 이천시 마장면 오천리 29221은(는) 검색에 실패했습니다
경기도

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y
0,의왕시,KT이자리에,오전동,333,2004-11-29,22,3,182,Y,경기도 의왕시 오전동 333,953557.781451749,1928041.69963887
1,의왕시,골드클래스,학의동,823-2,2019-05-03,16,8,420,N,경기도 의왕시 학의동 823-2,957109.109536009,1930673.93841674
2,의왕시,남양모란,오전동,358-19,1999-12-24,16,1,128,N,경기도 의왕시 오전동 358-19,953529.823673671,1928459.90041656
3,의왕시,뉴서울국화,오전동,838-3,1994-12-15,18,2,220,Y,경기도 의왕시 오전동 838-3,952799.765084006,1928974.01109952
4,의왕시,대명구름채,오전동,28,2004-12-21,23,6,518,Y,경기도 의왕시 오전동 28,952811.348472896,1929315.1928901


In [11]:
#5 seoul university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [12]:
#5 seoul university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in apt_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        apt_df.loc[_, "lng"] = x
        apt_df.loc[_, "lat"] = y

    except:
        pass
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y,lng,lat
0,의왕시,KT이자리에,오전동,333,2004-11-29,22,3,182,Y,경기도 의왕시 오전동 333,953557.781451749,1928041.69963887,126.975624,37.350249
1,의왕시,골드클래스,학의동,823-2,2019-05-03,16,8,420,N,경기도 의왕시 학의동 823-2,957109.109536009,1930673.93841674,127.015568,37.374146
2,의왕시,남양모란,오전동,358-19,1999-12-24,16,1,128,N,경기도 의왕시 오전동 358-19,953529.823673671,1928459.90041656,126.975282,37.354017
3,의왕시,뉴서울국화,오전동,838-3,1994-12-15,18,2,220,Y,경기도 의왕시 오전동 838-3,952799.765084006,1928974.01109952,126.967006,37.358614
4,의왕시,대명구름채,오전동,28,2004-12-21,23,6,518,Y,경기도 의왕시 오전동 28,952811.348472896,1929315.1928901,126.967115,37.361690


In [13]:
#5 seoul university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [15]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in apt_df.iterrows():
    try:
        folium.Marker(
            location=[apt_df.loc[idx, "lat"], apt_df.loc[idx, "lng"]],
            popup=apt_df.loc[idx, "공동주택명정보"],
            icon=folium.Icon(color="cadetblue", icon="house")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [16]:
#5 seoul university 
map

In [17]:
map.save("/aiffel/aiffel/000_aiffelthon_230111/3김연수/003_map_xml/apt_leecheon.html")

In [18]:
apt_df.to_csv("apt_leecheon.csv", index = False)